In [1]:
import numpy as np
import pandas as pd
from calibration.Calibration import Calibration 


In [3]:
# load rankings
reco_matrix = np.load("test/reco_matrix.npy")[0]
# scores = np.load("reco_matrix_mapped_scores.npy")[0]
print(reco_matrix.shape)
# print(scores.shape)

(943, 10)


In [5]:
import pickle

with open("test/score_dicts.pkl", "rb") as file:
    scores = pickle.load(file)

# print("Loaded List of OrderedDicts:", scores)

In [9]:
df_m = pd.read_csv(
    "./data/ml-100K/u.item",
    sep="|",
    names=[
        "movieID",
        "Name",
        "Date",
        "Video_Date",
        "IMDB_URL",
        "unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ],
    header=None,
    encoding="latin-1",
)
print(df_m.shape)
df_m = df_m[
    [
        "movieID",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ]
]

df_movies_mapped = pd.read_csv(
    "./data/ml-100K/i_id_mapping.csv",
    sep="\t",
    names=["movieID", "itemID"],
    header=None,
    encoding="latin-1",
)
movies = pd.merge(df_m, df_movies_mapped, how="inner", on="movieID")
movies = movies.drop(columns=["movieID"])
movies = movies.sort_values(by="itemID")

(1682, 24)


In [10]:
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
genre = movies[unique_genres]
item_features_numpy = genre.to_numpy()

users = pd.read_csv("./data/ml-100k/u_id_mapping.csv", sep="\t")

users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
user_features_numpy = users.to_numpy()
users

,Gender,userID
0,0,0
1,1,1
2,0,2
3,0,3
4,0,4
...,...,...
938,1,938
939,0,939
940,1,940
941,1,941


In [11]:
def create_genre_column(r):
    all_genres = [g for g in unique_genres if r[g] == 1]
    return "|".join(all_genres)


movies["genres"] = movies.apply(create_genre_column, axis=1)
movies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
240,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Comedy
300,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,Thriller|Mystery|Film-Noir|Crime
375,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Children's|Comedy
50,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,3,Romance|Western|War|Drama
344,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,Crime|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1344,Drama
1192,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1345,Comedy
1176,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1346,Drama
1261,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1347,Drama


In [12]:
item_ids = movies["itemID"].to_numpy()
item_ids

array([   0,    1,    2, ..., 1346, 1347, 1348])

In [15]:
scores.__len__()

943

In [ ]:
config = {"user_genre_dist_file": "./data/ml-100k/pgui.csv"}
calibration = Calibration(config, movies, 10, unique_genres, users)
calibration.get_improved_reco(reco_matrix, list(item_ids), scores)

[3]
[3, 0]
[3, 0, 1]
[3, 0, 1, 4]
[3, 0, 1, 4, 2]


ValueError: attempt to get argmax of an empty sequence

In [249]:
# tahsin kheya
# last modified 11/12/2024
import pandas as pd
import os
import numpy as np
import torch
import time
from logging import getLogger
import random
import numpy as np
from scipy.stats import entropy

class Calibration(object):
    def __init__(self, config, movies, top_k, unique_genres, users):
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(self.device)
        self.reco_distribution = []
        self.kl = []
        self.top_k = top_k
        self.gkl = []
        self.gender_df = users
        self.actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
        self.actual_genre_dist = self.actual_genre_dist.sort_values(by="userID")
        self.unique_genres = unique_genres
        self.config = config
        self.item_df = movies
        self.actual_distribution_without_id = self.actual_genre_dist.drop(
            columns=["userID"]
        )
        self.actual_distribution_without_id = self.actual_distribution_without_id.drop(
            columns=["user_timestamp_sum"]
        ).to_numpy()
        self.actual_distribution_gender = []

    def get_all_user_recommended_genre_dist(self, topk_reco):
        df_reco = pd.DataFrame(
            {
                "userID": np.repeat(np.arange(topk_reco.shape[0]), self.top_k),
                "itemID": topk_reco.flatten(),
                "rank": np.tile(np.arange(1, self.top_k + 1), topk_reco.shape[0]),
            }
        )
        df_reco["weight_factor"] = 1 / (df_reco["rank"]) ** 0.1
        merged_df = pd.merge(df_reco, self.item_df, on="itemID", how="inner")
        merged_df[self.unique_genres] = merged_df[self.unique_genres].div(
            merged_df[self.unique_genres].sum(axis=1), axis=0
        )
        merged_df[self.unique_genres] = (
            merged_df["weight_factor"].values[:, None] * merged_df[self.unique_genres]
        )
        summed_genre = merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        weight_total = merged_df.groupby("userID")["weight_factor"].sum()
        summed_genre["user_weight_factor"] = summed_genre["userID"].map(weight_total)
        summed_genre = merged_df.groupby("userID")[self.unique_genres].sum().reset_index()
        summed_genre = summed_genre[self.unique_genres].div(
            summed_genre["user_weight_factor"], axis=0
        )

        return summed_genre

    def get_recom_distribution(self, reco, uid):
        reco = np.array(reco)
        df_reco = pd.DataFrame(
            {
                "userID": np.repeat(uid, len(reco)),
                "itemID": reco.flatten(),
                "rank": np.tile(np.arange(1, len(reco) + 1), 1),
            }
        )
        df_reco["weight_factor"] = 1 / (df_reco["rank"]) ** 0.1
        merged_df = pd.merge(df_reco, self.item_df, on="itemID", how="inner")

        merged_df[self.unique_genres] = merged_df[self.unique_genres].div(
            merged_df[self.unique_genres].sum(axis=1), axis=0
        )
        merged_df[self.unique_genres] = (
            merged_df["weight_factor"].values[:, None] * merged_df[self.unique_genres]
        )
        summed_genre = merged_df.groupby("userID")[self.unique_genres].sum().reset_index()

        weight_total = merged_df.groupby("userID")["weight_factor"].sum()
        summed_genre["user_weight_factor"] = summed_genre["userID"].map(weight_total)

        summed_genre = summed_genre[self.unique_genres].div(
            summed_genre["user_weight_factor"], axis=0
        )
      
        
        return summed_genre[self.unique_genres].to_numpy()

    def get_kl_div(self, q_dist, p_dist, a, uid):
        qg_u = (1 - a) * q_dist + a * p_dist
        kl_div = entropy(p_dist, qg_u, base=10)

        return kl_div


    def compute_diversity_score(self, reco_items, uid, l, scores, b):
        alpha = 0.01
        sum_score = 0

        # -----------------the calibration term----------------------------------
       
        reco_dist = self.get_recom_distribution(reco_items, uid)[0]
        kl = self.get_kl_div(
             reco_dist,self.actual_distribution_without_id[uid], alpha, uid
        )

        # -----------------the calibration term----------------------------------

        # -----------------the fairness term----------------------------------
        # recommended dist mean for each gender
        male_user_ids = self.gender_df[self.gender_df["Gender"] == 0]["userID"]
        male_user_ids = male_user_ids.to_list()
        gender_genre_dist = self.actual_distribution_gender
        if uid in male_user_ids:
            compare_dist = gender_genre_dist[self.unique_genres].to_numpy()[1] #this is the female avg genre pref
        else:
            compare_dist = gender_genre_dist[self.unique_genres].to_numpy()[0]

        gender_kl = self.get_kl_div(reco_dist,compare_dist, alpha, uid)

        # -----------------the fairness term----------------------------------

        for r in range(len(reco_items)):
            sum_score += scores[reco_items[r]]
           
        return (1 - l - b) * sum_score - l * kl - b * gender_kl


    def get_improved_reco(self, top_items, items, scores):
        return self.get_new_recommendations(
            reco=top_items, scores=scores, all_items=items
        )


    def get_gender_genre_dist(self):
        actual_dist_gender = pd.merge(
            self.actual_genre_dist, self.gender_df, on="userID"
        )
        gender_genre_weights_a = actual_dist_gender.groupby("Gender")[
            self.unique_genres
        ].mean()
        self.actual_distribution_gender = gender_genre_weights_a.sort_index()

    def get_new_recommendations(self, reco, scores, all_items):
        """reco is 6040x50 and scores is 6040x3416"""
        self.get_gender_genre_dist()
        b = 0.69  # beta for the fairness term
        l = 0.29  # lambda for the calibration term
        all_users = []
        top_k = self.top_k
        for u in range(0, 3):
            remaining_items = list(range(20))
            u_calibrated = []
            for k in range(top_k):
                diversity_scores = [
                    self.compute_diversity_score(
                        u_calibrated + [i], u, l, scores[u], b
                    )
                    for i in remaining_items
                ]
                max_index = np.argmax(diversity_scores)

                best_item = remaining_items[max_index]
                u_calibrated.append(best_item)
                remaining_items.pop(max_index)
                print(u_calibrated)

            all_users.append(u_calibrated)

        return np.array(all_users)

In [250]:
config = {"user_genre_dist_file": "../data/ml-100k/pgui.csv"}
# movies = movie df
calibration = Calibration(config, movies, 10, unique_genres, users)
print(calibration.get_improved_reco(reco_matrix, list(item_ids), scores))

[17]
[17, 16]
[17, 16, 10]
[17, 16, 10, 1]
[17, 16, 10, 1, 3]
[17, 16, 10, 1, 3, 2]
[17, 16, 10, 1, 3, 2, 8]
[17, 16, 10, 1, 3, 2, 8, 6]
[17, 16, 10, 1, 3, 2, 8, 6, 0]
[17, 16, 10, 1, 3, 2, 8, 6, 0, 9]
[10]
[10, 3]
[10, 3, 1]
[10, 3, 1, 9]
[10, 3, 1, 9, 6]
[10, 3, 1, 9, 6, 7]
[10, 3, 1, 9, 6, 7, 0]
[10, 3, 1, 9, 6, 7, 0, 8]
[10, 3, 1, 9, 6, 7, 0, 8, 16]
[10, 3, 1, 9, 6, 7, 0, 8, 16, 18]
[10]
[10, 3]
[10, 3, 1]
[10, 3, 1, 0]
[10, 3, 1, 0, 4]
[10, 3, 1, 0, 4, 7]
[10, 3, 1, 0, 4, 7, 6]
[10, 3, 1, 0, 4, 7, 6, 12]
[10, 3, 1, 0, 4, 7, 6, 12, 8]
[10, 3, 1, 0, 4, 7, 6, 12, 8, 9]
[[17 16 10  1  3  2  8  6  0  9]
 [10  3  1  9  6  7  0  8 16 18]
 [10  3  1  0  4  7  6 12  8  9]]


In [ ]:
movies[movies["itemID"] == 96]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
206,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,96,Comedy|Horror


In [ ]:
p = np.array([0.1, 0.3, 0.6])
q = np.array([0.3, 0.4, 0.3])
# q˜(д|u) = (1 − α) · q(д|u) + α · p(д|u)
q_t = (1 - 0.01) * q + 0.01 * p
print(q_t)

[0.298 0.399 0.303]


In [ ]:
0.99 * 0.3 + 0.01 * 0.1

0.298

In [ ]:
sum([0.298, 0.399, 0.303])

1.0

In [ ]:
sum(
    [
        0.0,
        0.01923077,
        0.0,
        0.11965812,
        0.0,
        0.02136753,
        0.0,
        0.00641026,
        0.0,
        0.02564101,
        0.5769231,
        0.02777778,
        0.01282051,
        0.0,
        0.0,
        0.0,
        0.01495726,
        0.0,
        0.17521365,
    ]
)

0.99999999

In [ ]:
item_index = movies.index.get_loc(0)
item_index

24

In [ ]:
reco = [1, 23, 4]
reco_2 = np.array(reco)
reco_2.flatten()

array([ 1, 23,  4])

In [ ]:
nonzero_indices = np.where(p_dist != 0)

In [ ]:
nonzero_indices

(array([0, 2]),)